In [ ]:
WITH Canadian_PEP AS (
    -- 1. Grab all valid Cost Center IDs from Canadian employees
    SELECT 
        SUBSTRING_INDEX(TRIM(Costcenter), ' ', 1) AS Extracted_CC
    FROM hive_metastore.ra_adido_2025.employee_pep_list_as_of_oct312025
    WHERE TRIM(Region) = 'Canada'
)

-- 2. Join to mapping table and group by Assessable Unit
SELECT 
    m.`Assessable Unit ID and Name` AS `Assessable Unit`,
    
    -- If the join found at least one matching officer for this AU, flag it 'Yes'. Otherwise, 'No'.
    CASE 
        WHEN COUNT(p.Extracted_CC) > 0 THEN 'Yes' 
        ELSE 'No' 
    END AS `Has Canadian Officer?`

FROM cost_center m

LEFT JOIN Canadian_PEP p
    -- Smart Join: Adds a '0' only if the ID is exactly 3 characters long
    ON CASE 
        WHEN LENGTH(TRIM(CAST(m.`Cost Center` AS STRING))) = 3 
        THEN CONCAT('0', TRIM(CAST(m.`Cost Center` AS STRING))) 
        ELSE TRIM(CAST(m.`Cost Center` AS STRING)) 
       END = p.Extracted_CC

-- Filter out any totally blank rows from your mapping file
WHERE m.`Assessable Unit ID and Name` IS NOT NULL

-- Roll everything up so there is only one row per Assessable Unit
GROUP BY 
    m.`Assessable Unit ID and Name`